# Alpaca LLaMa 7B LoRa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/tloen/alpaca-lora.git

In [ ]:
%cd alpaca-lora/

/content/alpaca-lora


In [ ]:
!pip install -q datasets loralib sentencepiece

!pip uninstall transformers
!pip install git+https://github.com/huggingface/transformers

!pip install -q git+https://github.com/huggingface/peft.git

In [ ]:
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 18.8 MB/s eta 0:00:00


## Data Check

In [ ]:
from datasets import load_dataset
from transformers import LlamaTokenizer


#Initializing tokenizer for Datacheck
tokenizer = LlamaTokenizer.from_pretrained("wxjiao/alpaca-7b", add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


#ADJUST FILEPATH
data = load_dataset("json", data_files="/path/to/therapy_dataset.json")


def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["instruction"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""


data = data.map(lambda data_point: {"prompt": tokenizer(generate_prompt(data_point))})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

## Fine Tuning

In [ ]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, LlamaTokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model

In [ ]:
MICRO_BATCH_SIZE = 24  # Reduce this to decrease VRAM requirements
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3  # paper uses 3
LEARNING_RATE = 2e-5  # from the original paper
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [ ]:
model = LlamaForCausalLM.from_pretrained(
    "wxjiao/alpaca-7b",
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = LlamaTokenizer.from_pretrained(
    "wxjiao/alpaca-7b", add_eos_token=True, trust_remote_code=True
)

model = prepare_model_for_int8_training(model)


In [ ]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
data = load_dataset("json", data_files="/path/to/therapy_dataset.json")


In [ ]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Response:
{data_point["output"]}"""


data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir="/path/to/checkpoint/output/lora-alpaca-counseling-checkpoint",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

model.save_pretrained("/path/to/save/model/to/lora-alpaca-counseling-finished")

## Generation

In [ ]:
# LOADING MODEL AND APPLYING LORA WEIGHTS 

from peft import PeftModel
from transformers import GenerationConfig
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, LlamaTokenizer


tokenizer = LlamaTokenizer.from_pretrained("wxjiao/alpaca-7b")

model = LlamaForCausalLM.from_pretrained(
    "wxjiao/alpaca-7b",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/lora-alpaca-counseling-finished")


In [ ]:
#SINGLE TURN MODEL INFERENCE

## To try out the model change the instruction text from "I am feeling pretty down lately and am not really sure what to do." to anything you wish. 

PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
I am feeling pretty down lately and am not really sure what to do.
### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=128,
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))